In [98]:
import pandas as pd
import numpy as np
from llms import gemini
from llms import chatGPT
import re
x_chat = chatGPT()
x_gemini = gemini()

In [108]:
df = pd.read_csv("../output/data.csv")
df = df[df["num employees"]=="1-10"]
df = df[df["headquarters location"].str.contains("United States")]
df = df[pd.to_datetime(df['founded date'])>'11/30/2022']
sample_df = df.sample(n=1, random_state=1)
sample_df = sample_df.reset_index(drop=True)
sample_df.head()

,Unnamed: 0,organization name,num employees,founded date,description,industries,headquarters location,description.1,cb rank,postal code,...,email,hub tag,phone num,num of sub org,industry groups,total funding amt,company type,estimated rev,description_all,industries_parsed
0,19076,NonprofitsHQ,1-10,2023-02-03,NonprofitsHQ is a powerful suite of AI-powered...,"B2B, SaaS","San Antonio, Texas, United States",B2B SaaS for nonprofits,"476,474",78205,...,hello@nonprofitshq.com,—,—,—,"Other, Software",—,For Profit,—,NonprofitsHQ is a powerful suite of AI-powered...,"['B2B', 'SaaS']"


In [109]:
df_exp = pd.read_csv('../output/onet/gpt_exposure_embeddings.csv')
df_exp.head()

,Unnamed: 0,index,O*NET-SOC Code,Title,Task ID,Task,human_labels,gpt_4_exposure,gpt_4_automation,human_numeric,gpt_4_numeric,gpt_4_automation_numeric,description,Description,Embeddings
0,0,0,11-2022.00,Sales Managers,1,Resolve customer complaints regarding sales an...,E1,E2,T2,1.0,0.5,0.50,Sales managers that resolve customer complaint...,Sales managers that resolve customer complaint...,"[0.016441883519291878, -0.031463537365198135, ..."
1,1,1,11-2022.00,Sales Managers,2,Monitor customer preferences to determine focu...,E2,E2,T2,0.5,0.5,0.50,Sales managers that monitor customer preferenc...,Sales managers that monitor customer preferenc...,"[0.014131995849311352, -0.0319337397813797, -0..."
2,2,2,11-2022.00,Sales Managers,4,Determine price schedules and discount rates.,E2,E2,T3,0.5,0.5,0.75,Sales managers that determine price schedules ...,Sales managers that determine price schedules ...,"[-0.012649821117520332, -0.019506914541125298,..."
3,3,3,11-2022.00,Sales Managers,5,Review operational records and reports to proj...,E2,E2,T3,0.5,0.5,0.75,Sales managers that review operational records...,Sales managers that review operational records...,"[0.0030783088877797127, -0.01418742723762989, ..."
4,4,4,11-2022.00,Sales Managers,7,Confer or consult with department heads to pla...,E2,E2,T1,0.5,0.5,0.25,Sales managers that confer or consult with dep...,Sales managers that confer or consult with dep...,"[0.020180119201540947, -0.02971957065165043, -..."


In [110]:
df_exp.description.iloc[0]

'Sales managers that resolve customer complaints regarding sales and service.'

In [121]:
import time

generated_description_prompt = """
Your role is to describe $company ($website)'s product in two sentences. Then, provide a confidence interval on scale on 1-10 on how sure you are about the response.

EXAMPLES BELOW:
_________________________
Company: Genmo
Website: https://www.genmo.ai/
Description: Genmo is a free tool that creates videos and images using artificial intelligence. Genmo to generate creative text formats of text content, like poems, code, scripts, musical pieces, email, letters, etc.

Company: OnePane
Website: https://www.onepane.ai/
Description: Onepane is a company that offers an AI companion for enhanced DevOps & SRE efficiency. Onepane offers a GenAI solution  providing unparalleled unified insights and control over your Cloud resources. Onepane helps with root cause analysis, cloud governance, and optimization strategies.
________________________

YOUR TURN:
Company: $company
Wesbite: $website
Current Description: $description
________________________
Description:
Confidence Interval:
Reasoning:
________________________

"""



In [146]:
parsed_description_prompt = """
Your role is to describe what jobs/tasks and industries that $company is automating. Please refer to examples. Then, provide a confidence interval (1-10) on scale on 1-10 on how sure you are about the response.

Be specific about people using tool. If it applies to many different people (over 20% of population), then say everyone.
DO NOT INCLUDE AN EXPLANATION

EXAMPLES BELOW 
_________________________
Company: Petville
Website: petville.co/pricing/biz
Description: Petville Global is a B2B CRM SaaS platform that utilizes advanced technologies like AI/ML and neural net to streamline and expand operations for pet businesses and veterinary clinics both locally and globally. The platform offers deep data analytics and marketing tools, helping businesses save an average of 22% on CRM and vet tech costs.
Tasks/Jobs: Data analysis, Marketing automation, Appointment scheduling, Inventory management
Industry: Customer management
People Using Tool: Vetinarians, Pet Businesses

Company: Thunder
Wesbite: thundercompute.com
Description: Thunder is a decentralized, peer-to-peer cloud computing platform designed to democratize access to GPUs and address the persistent GPU shortage. It provides a solution for individuals and businesses seeking high-performance computing power, enabling them to leverage unused GPU resources from around the world.
Tasks/Jobs: GPU resource allocation, Access to unused GPUs, Distributed computing tasks 
Industry: Cloud Computing
People Using Tool: Developers, GPU Owners


Company: InputAI
Wesbite: inputai.com/
Description: InputAI is an advanced platform that offers over 1000 AI templates powered by OpenAI GPT and ChatGPT technologies. It is designed to streamline and enhance the user's interaction with AI, providing a wide range of templates for various applications, from chatbots to content generation.
Tasks/Jobs: Chatbot development, Content creation
Industry: No-code AI platform
People Using Tool: Everyone
________________________

YOUR TURN:
________________________
Company: $company
Wesbite: $website
Description: $description
________________________
Tasks/Jobs (comma separated list of 4, short):
Industry (1 item):
People Using Tool (comma separated list, one/two words each): 
________________________
Confidence Interval:
Reasoning:
________________________
"""


In [150]:
examples_prompt = """
Your role is to give me 3 two sentence example that would be using the product of $company. Do not use name of company in description. Keep it broad.
The goal is to give the a detailed description of the job that the tool automates and who performs that job and when.Then, provide a confidence interval (1-10) on scale on 1-10 on how sure you are about the response.

IF THE PERSON AUTOMATED IS NOT DOING A JOB THAT IS NOT DONE AT WORK THEN WRITE "NOT_ONET" IN FRONT OF IT.

EXAMPLE: 
************************
Company: DreamGenerator.ai
Website: DreamGenerator.ai
DreamGenerator.ai is an innovative platform that utilizes generative AI to transform user ideas into stunning and diverse images. It offers a unique creative experience, encouraging users to experiment with prompts and share their AI-generated art, while also providing an opportunity to receive recognition through likes and shares.
Tasks/Jobs: Image generation, Artistic exploration, Prompt engineering, Social media sharing
Industry: Creative arts
People Using Tool: Artists, Designers, Everyone
________________
Example 1: A writer uses generates images that will inspire ideas for their new book by typing in fun prompts into the website.
Job automated 1: Writer has to generate create ideas for book.
Example 2: A social media marketer creates eye-catching visuals for their summer fitness campaign, like "people doing yoga on a sunrise beach," then uses them for engaging social media posts.
Job automated 2: Graphic designer createsn images for summer fitness brand campaign.
Example 3: An architect generates images that spark ideas to for the new building. They type prompts like "skyscraper covered in living walls" and "underwater hotel with transparent pods," generating visuals to inspire their sustainable architecture concepts.
Job autoamted 3: Architect generates ideas for new building.
________________

Company: Cozy Ventures
Wesbite: https://cozy.ventures/
Current Description: Cozy Ventures is a boutique software development company that provides startups with innovative digital solutions to accelerate their growth. Composed of a team of seasoned engineers and designers, they specialize in creating custom software tailored to meet the unique needs of each client.
Tasks/Jobs: Software development,  UI/UX design,  Project management,  Technical consulting
Industry:  Software development
People Using Tool:  Startups,  Entrepreneurs
________________
Example 1: A food delivery startup develop a custom app that integrates real-time tracking, route optimization, and communication features. 
Person automated 1: A food deliver manager is in charge of real-time tracking, route optimization, and communication for food delivery employees.
Example 2: An e-commerce company build a custom recommendation engine, powered by AI, that analyzes user behavior and preferences to offer relevant product suggestions.
Person automated 2: A market research analyst for a ecommerce company analyzes user behavior and preferences to offer relevant product suggestions
Example 3: A social media platform develops an app a sophisticated content moderation system that leverages machine learning algorithms to identify and flag inappropriate content in real-time. 
Person automated 3: A social media content moderator identifies and flag inappropriate content in real-time.
________________


Company: PnPAI
Website: www.pnpai.co
PnPAI is an AIaaS platform that standardizes the implementation of AI across various industries, providing pre-built solutions that can be easily integrated without the need for in-house AI expertise. Catering to businesses of all sizes, PnPAI offers industry-focused solutions for sectors like ecommerce, retail, healthcare, and finance, making AI an accessible and integral part of decision-making and growth strategies.
Tasks/Jobs:  Data analysis, Predictive modeling, Automation, Optimization
Industry: AIaaS (AI as a Service)
People Using Tool: Businesses, Professionals 
________________
Example 1: A retail manager uses PnPAI to analyze customer purchasing patterns and predict future demand for specific products. They then adjust inventory levels and optimize product placement in stores based on the AI-generated insights.
Person automated 1: A data analyst for a retail manager performs inventory management assestment to predict future custumer demand.
Example 2: An insurance company uses PnPAI to automate the risk assessment process for new clients. The AI system analyzes various data points, including credit history, driving records, and health information, to quickly determine insurance premiums and coverage options. 
Person automated 2: A risk assesser for an insurance agency uses data to perform risk assement for new clients. 
Example 3: A healthcare provider leverages PnPAI to personalize treatment plans for patients with chronic diseases. The AI analyzes patient medical history, current symptoms, and lifestyle factors to recommend optimal medication dosages and treatment strategies. 
Person automated 3: A data analysts for a healthcare company uses patient data to form personalized treatment plan creation. 
________________

************************
YOUR TURN:
Company:$company
Website: $website
Current Description: $description
$parsed_description
________________
Example 1: 
Person automated 1:
Example 2: 
Person automated 2:
Example 3:
Person automated 3:
________________
Confidence Interval:
Reasoning:
________________
"""


In [163]:
class prompting():
    def __init__(self):
        pass

    def extract_data1(self,text):
        text = text.replace("\n"," ").replace("  "," ").replace("*","").replace("Description:","")
        pattern = r"^(.*?)\s*Confidence Interval: (\d+)\s+Reasoning: (.*)$"
        match = re.search(pattern, text, re.DOTALL)
        vals = []
        if match:
             for x in range(1,4):
                 vals.append(match.group(x).strip())
        return vals

    
    def iterate(self, df, prompt_template, type, args,i=0):
        results = []
        
        for index, row in list(df.iterrows())[i:]:
            prompt = prompt_template
            while True:
                try:
                    name = row['organization name']
                    website = row['website']
                    print(f"Processing {index}: {name}, {website}")
                    
                    for arg in args: 
                        prompt = prompt.replace(f"${arg[0]}", row[arg[1]])
                    result = x_gemini.ask(prompt)
                    
                    print(f"Result: {result}\n")
                    result_vals = self.extract_data1(result)
                    print(result_vals)
                    results.append([name]+result_vals)
                    break 
                
                except Exception as e:
                    print(f"Error processing {index}, {row['organization name']}: {e}")
                    time.sleep(20)  
        if type in df.columns:
            df = df.drop(columns=[type,type+"_conf_interval", type+"_conf_interval_resoning"])
        self.results_df = pd.DataFrame(results, columns=["organization name", type, type+"_conf_interval", type+"_conf_interval_resoning"])
        df = df.merge(self.results_df, on='organization name', how='left')
        return df

prompting_class = prompting()


In [ ]:
# args = [["company","organization name"],["website","website"], ["description","description_all"]]
# sample_df = prompting_class.iterate(sample_df, generated_description_prompt, "generated_description", args)


In [ ]:
def extract_data(text, pattern):
    matches = re.findall(pattern, text, flags=re.DOTALL)
    vals = []
    for match in matches:
        vals.extend([m.strip() for m in match if m != ''])
    return vals




In [ ]:
args = [["company","organization name"],["website","website"], ["description","generated_description"]]
sample_df = prompting_class.iterate(sample_df, parsed_description_prompt, "parsed_description", args)

pattern = r'Tasks/Jobs:\s*(.*?)\s*Industry:\s*(.*?)\s*People Using Tool:\s*(.*)'
sample_df[['Tasks/Jobs', 'Industry', 'People Using Tool']] = list(sample_df.parsed_description.apply(lambda text: extract_data(text,pattern)))


In [152]:
args = [["company","organization name"],["website","website"], ["description","generated_description"], ["parsed_description","parsed_description"]]
sample_df = prompting_class.iterate(sample_df, examples_prompt, "examples", args)


pattern = r"(Example \d+: .*?)(?= Example \d+:|Person automated \d+:|$)|(Person automated \d+: .*?)(?= Example \d+:|Person automated \d+:|$)"
sample_df[['Example1','Job1','Example2','Job2','Example3','Job3']] = list(sample_df.examples.apply(lambda text: extract_data(text,pattern)))


Processing 0: NonprofitsHQ, www.nonprofitshq.com
Result: ## NonprofitsHQ Examples:

**Example 1:** A development director uses the platform to segment their donor base, sending personalized appeals to different groups based on their giving history and interests. They leverage AI-powered insights to optimize their fundraising campaigns and increase donor retention.
**Person automated 1:** NOT_ONET A development director performs donor segmentation and creates individualized appeals based on giving history and interests.

**Example 2:** A volunteer coordinator uses the platform to automate communication with volunteers, sending targeted updates and reminders about upcoming events and volunteer opportunities. 
**Person automated 2:** NOT_ONET A volunteer coordinator sends targeted updates and reminders about upcoming events and volunteer opportunities. 

**Example 3:** A grants manager utilizes the platform to track grant proposals, deadlines, and reporting requirements, ensuring timely a

In [ ]:
sample_df.loc[:, ["Job1_ONET", "Job2_ONET", "Job3_ONET"]] = 'N/A'

results = []
for i, x in sample_df.iterrows():
    name = x["organization name"]
    print(f"Processing {str(i), len(results)}: {name}")
    job1_embedding = x_chat.get_embedding(x["Job1"])
    job2_embedding = x_chat.get_embedding(x["Job2"])
    job3_embedding = x_chat.get_embedding(x["Job3"])
    results.append([name,job1_embedding, job2_embedding, job3_embedding])

df = pd.DataFrame(results, columns=['name', "Job1_embedding","Job2_embedding","Job3_embedding"])
sample_df = pd.concat([sample_df, df], axis=1)

